In [2]:
#Percy's HOBO processing code, with small modifications made by Eren 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,scatter,savefig,figure,colorbar,suptitle,close,show,subplot,rgrids
import pylab
from matplotlib.mlab import csv2rec
import matplotlib as mpl
import datetime
from matplotlib.dates import DateFormatter
from time import mktime
from scipy.interpolate import interp1d
import pickle

In [3]:
# calibration relationships
calnew = csv2rec('/Users/Eren/Desktop/data/HOBO/linregressions.csv')
calK = csv2rec('/Users/Eren/Desktop/data/HOBO/calibrations_Kevinsensors.csv')
#These are record arrays. Each row contains the regression coefficients for a particular sensor. The entries in the rows
#are sensor#, mT(i.e. the slope coeffient for temperature), bT (i.e the intercept coefficient for temperature), mRH, and
#finally bRH. This is documented in the dtype row which shows up at the bottom.
#Below is how you look at the header of a record array:
calnew[0:10]

rec.array([(32, 0.953976046759, 0.64074421949, 1.01009562711, 0.820251646579),
 (33, 0.93028776209, 1.20901119849, 0.984301625197, 1.40979323766),
 (34, 0.990206058501, 0.36112728793, 0.965737323224, 0.663595892954),
 (35, 0.988577614232, 0.485784725522, 0.95853246459, 0.863696131504),
 (36, 1.03663052043, -0.547961711482, 1.00619079225, -0.333523569232),
 (37, 1.03843884483, -0.83563849316, 1.01428928775, -0.362569647039),
 (38, 1.03702566634, -0.796974732731, 1.01254902482, -0.381930208069),
 (39, 1.0567588972, -1.14557548562, 1.0164639295, -0.502797080886),
 (40, 1.06455479323, -1.36369645284, 1.05161790422, -1.58042902666),
 (41, 1.03886092231, -0.978935812207, 0.992379717168, 0.0548943797267)], 
          dtype=[('sensor', '<i8'), ('mt', '<f8'), ('bt', '<f8'), ('mrh', '<f8'), ('brh', '<f8')])

In [4]:
#In this section, we are creating a dictionary object that contains the file path tails relevant to each sensor number.
#In the loop that uses these file path tails, there is a place to specify the file path head which is common to each 
#but local to your computer, so look out for that later. To get this to run on your computer you need to change it.

#Note also that you can relate sensor number to position on the hill by looking in both HoboDeviceProperties.xlsx and
#SensorCartoonMap.pptx
files = dict()
# 795 = 1-3
# files['795'] = ['/25 August 2010/station1-3.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373795.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373795_0.csv','/6 Feb 2012/2373795.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373795.csv',\
#     '/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373795_0.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373795.csv']
# # 796 = 2-4
# files['796'] = ['/25 August 2010/station2-4.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373796_0.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373796_1.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373796_2.csv',\
#     '/6 Feb 2012/2373796.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373796.csv','/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373796.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373796.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373796_0.csv']
# # 797 = 2-1
# files['797'] = ['/25 August 2010/station2-1.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373797_0.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373797_1.csv','/6 Feb 2012/2373797.csv',\
#     '/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373797.csv','/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373797.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373797.csv']
# # 798 = 1-1
# files['798'] = ['/25 August 2010/station1-1.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373798.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373798_0.csv','/6 Feb 2012/2373798.csv',\
#     '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373798.csv','/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373798_0.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373798.csv']
# # 799 = S-1
# files['799'] = ['/25 August 2010/station3-1.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373799_0.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373799_1.csv','/6 Feb 2012/2373799.csv',\
#     '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373799.csv','/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373799.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373799.csv']
# # 800 = 1-4
# files['800'] = ['/25 August 2010/station1-4.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373800.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373800_0.csv','/6 Feb 2012/2373800.csv',\
#     '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373800.csv','/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373800_0.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373800.csv']
# # 803 = S-3
# files['803'] = ['/25 August 2010/station3-3.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373803_0.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373803_1.csv','/6 Feb 2012/2373803.csv',\
#     '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373803.csv','/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373803.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373803.csv']
# # 804 = 2-3
# files['804'] = ['/25 August 2010/station2-3.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373804_0.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373804_1.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373804_2.csv',\
#     '/6 Feb 2012/2373804.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373804.csv',\
#     '/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373804.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373804.csv']
# # 807 = 1-2
# files['807'] = ['/25 August 2010/station1-2.csv','/ShuttleReadout12_10_10_10_02_57_AM_PST/2373807.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373807_0.csv','/6 Feb 2012/2373807.csv',\
#     '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373807.csv','/ShuttleReadout05_07_12_02_07_41_PM_GMT-07_00/2373807_0.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373807.csv']
# # 808 = S-2
# files['808'] = ['/25 August 2010/station3-2.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373808_0.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373808_1.csv','/6 Feb 2012/2373808.csv',\
#     '/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373808.csv','/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373808.csv',\
#     '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373808.csv']
# # 809 = S-4
# files['809'] = ['/6 May 2010/Station3-4.csv','/25 August 2010/station3-4.csv',\
#     '/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373809_0.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373809_1.csv',\
#     '/6 Feb 2012/2373809.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373809.csv',\
#     '/ShuttleReadout05_07_12_04_22_10_PM_GMT-07_00/2373809.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373809.csv']
# # 810 = 2-2
# files['810'] = ['/25 August 2010/station2-2.csv','/ShuttleReadout10_22_10_08_47_13_AM_PDT/2373810_0.csv',\
#     '/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373810_1.csv','/ShuttleReadout09_07_11_05_38_23_PM_PDT/2373810_2.csv',\
#     '/6 Feb 2012/2373810.csv','/ShuttleReadout03_16_12_12_04_52_PM_GMT-07_00/2373810.csv',\
#     '/ShuttleReadout05_07_12_04_56_23_PM_GMT-07_00/2373810.csv','/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/2373810.csv']
# 40 = ILean 30m
files['40'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042540_0_mod.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042540.csv']
# 52 = ILean 25m
files['52'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042552_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042552.csv']
# 44 = ILean 20m
files['44'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042544_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042544.csv']
# 45 = ILean 15m
files['45'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042545_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042545.csv']
# 43 = ILean 10m
files['43'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042543_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042543.csv']
# 36 = ILean 5m
files['36'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042536_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042536.csv']
# 47 = FT 28m
files['47'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042547.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042547.csv']
# 56 = FT 22m
files['56'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042556.csv',\
    '/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042556_0.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042556.csv']
# 35 = FT 17m
files['35'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042535.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042535.csv']
# 41 = FT 12m
files['41'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042541.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042541.csv']
# 33 = FT 7m
files['33'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042533.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042533.csv']
# 49 = FT 2m
files['49'] = ['/ShuttleReadout05_07_12_03_45_25_PM_GMT-07_00/HydroWatch_10042549.csv',\
    '/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042549.csv']
# 62 = Ursula 25 m
files['62'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042562.csv']
# 58 = Ursula 20 m
files['58'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042558.csv']
# 54 = Ursula 15 m
files['54'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042554.csv']
# 61 = Ursula 10 m
files['61'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042561.csv']
# 42 = Ursula 5 m
files['42'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042542.csv']
# 59 = SMM 17.5 m
files['59'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042559.csv']
# 32 = SMM 15 m
files['32'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042532.csv']
# 60 = SMM 10 m
files['60'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042560.csv']
# 48 = SMM 5 m
files['48'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042548.csv']
# 34 = SMM 2.5 m
files['34'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042534.csv']
# 63 = SMU 17.5 m
files['63'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042563.csv']
# 37 = SMU 15 m
files['37'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042537.csv']
# 66 = SMU 10 m
files['66'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042566.csv']
# 65 = SMU 7.5 m
files['65'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042565.csv']
# 64 = SMU 5 m
files['64'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042564.csv']
# 50 = Freddie 30 m
files['50'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042550.csv']
# 51 = Freddie 24 m
files['51'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042551.csv']
# 53 = Freddie 18 m
files['53'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042553.csv']
# 39 = Freddie 12 m
files['39'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042539.csv']
# 46 = Freddie 6 m
files['46'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042546.csv']
# 38 = road between well 6 and well 5
files['38'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042538.csv']
# 57 = near well 15, top of ridge
files['57'] = ['/ShuttleReadout09_16_12_01_13_41_AM_GMT-07_00/HydroWatch_10042557.csv']


In [5]:
# Singling out the late sensors: list of sensors placed in the field on May 7 2012
latesensors = ['62','58','54','61','42','59','32','60','48','34','63','37','66','65','64','50','51','53','39','46']


In [6]:
#Preparing for a loop over all sensors; make a list of all sensors
sensors = files.keys()
sorted(sensors)

['32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66']

In [7]:
#preparing for a loop over all sensors
#make some empty dictionary objects for datetime (dt), Temperature (T), and Relative Humidity (RH) data
dt = dict()
T = dict()
RH = dict()

In [8]:
#start the loop
for ss in sensors:
    print(ss) #to show which one we are on
    
    dttmp = np.array([]) #make empty temporary arrays for each variable
    Ttmp = np.array([])
    RHtmp = np.array([])
    
     # loop through files and (1) make record arrays out of their data, (2) delete unecessary rows (3) adjust time
        #if necessary, (4) calibrate data 
    for ff in files[ss]:
        # read file
        data = csv2rec('/Users/Eren/Desktop/data/HOBO/Data'+ff,skiprows=1,comments='!') #This needs to be changed to
        #reflect the local storage path to run on another computer.
        fields = data.dtype.names
        mask = (data.end_of_file=='')#This gets rid of the end of file row by only selecting rows where the end of file
        #field is empty.  
        if 'coupler_detached' in fields:
            mask = mask & (data.coupler_detached=='') #This gets rid of the row that spawned when the coupler detached 
            #during a logging of the data. 
        if 'coupler_attached' in fields:
            mask = mask & (data.coupler_attached=='')#This gets rid of the row that spawned when the coupler attached
            #during a logging of the data. 
        if 'stopped' in fields:
            mask = mask & (data.stopped=='') #This gets rid of the row where the data collection stopped
        Ttmp = np.append(Ttmp,data['temp_c'][mask])
        RHtmp = np.append(RHtmp,data.rh_[mask])
        
        # adjust time
        if len(ss)==3:
            # Kevin's sensors are on daylight savings time - subtract an hour
            dttmp = np.append(dttmp,(data.date_time_gmt0700[mask]-datetime.timedelta(seconds=3600))) 
        elif len(ss)==2:
            dttmp = np.append(dttmp,data.date_time_gmt0800[mask])
            
    # exclude non-field data for sensors placed in field on May 7 2012
    if ss in latesensors:
        mask = dttmp>=datetime.datetime(2012,5,7,17,0,0)
        dttmp = dttmp[mask]
        Ttmp = Ttmp[mask]
        RHtmp = RHtmp[mask]
    # calibrate
    if len(ss)==3: #three-digit sensors use calK calibrations
        mt = calK.mt[calK.sensor==ss]
        bt = calK.bt[calK.sensor==ss]
        mrh = calK.mrh[calK.sensor==ss]
        brh = calK.brh[calK.sensor==ss]
    elif len(ss)==2: #two-digit sensors use calnew calibrations
        mt = calnew.mt[calnew.sensor==ss]
        bt = calnew.bt[calnew.sensor==ss]
        mrh = calnew.mrh[calnew.sensor==ss]
        brh = calnew.brh[calnew.sensor==ss]
    
    Ttmp = mt*Ttmp+bt
    RHtmp = mrh*RHtmp+brh

    # adjust rel hum
    RHtmp[RHtmp>100.] = 100. #Make is so it can't go over 100%

    if ss=='803':
        RHtmp = RHtmp+np.nan  # replace bad RH sensor data with nans
        
    # save date, temp, rel hum
    dt[ss] = dttmp
    T[ss] = Ttmp
    RH[ss] = RHtmp
    
#Create a dictionary object to store all the data
data = dict() 
data['dt'] = dt
data['T'] = T
data['RH'] = RH
pickle.dump(data, open('HOBOdata.p','wb'))

40


/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: DeprecationWarning: using a boolean instead of an integer will result in an error in the future
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:50: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:50: DeprecationWarning: using a boolean instead of an integer will result in an error in the future
/Users/Eren/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:51: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/Users/Eren/anaconda/lib/python3.5/site-packages/ipyker

34
35
32
42
43
47
51
45
60
36
37
48
53
64
44
38
56
59
63
39
54
41
58
33
66
52
57
49
61
50
46
62
65
